In [1]:
import os
%pwd


'f:\\ml_projects\\playing_card_detection\\notbooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml_projects\\playing_card_detection'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    data_validation_status_file:Path
    data_validation_required_file:list
    unzip_dir:Path

In [4]:
from Playing_card_detection.constants import *
from Playing_card_detection.utils.common import read_yaml_file,create_directories
from pathlib import Path

class ConfigurationManager():
    def __init__(self,config_filepath = CONFIG_FILEPATH) -> None:
        self.config = read_yaml_file(config_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        temp_config = self.config.data_validation
        create_directories([temp_config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = Path(temp_config.root_dir),
            data_validation_status_file = Path(temp_config.data_validation_status_file),
            data_validation_required_file = temp_config.data_validation_required_file,
            unzip_dir = Path(self.config.data_ingestion.unzip_dir)
        )
        return data_validation_config

In [10]:
from Playing_card_detection.utils.exception import CustomException
from Playing_card_detection.utils.logger import logger
import os,sys

class DataValidation:
    def __init__(self,config:DataValidationConfig) -> None:
        self.config = config
    
    def do_validation(self):
        try:
            validation_status = None
            file_that_exist = os.listdir(self.config.unzip_dir)
            required_files_list = self.config.data_validation_required_file
            for file in required_files_list:
                if file not in file_that_exist:
                    validation_status = False
                    with open(self.config.data_validation_status_file,'w') as f:
                        f.write(f"Validation Status == {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.data_validation_status_file,'w') as f:
                        f.write(f"Validation Status == {validation_status}")
            if validation_status:
                logger.info("Data Validation Succeed")
            else:
                logger.info("Data Validation Failed")
        except Exception as e:
            raise CustomException(e,sys)

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.do_validation()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-01-07 10:10:55,303 ] - 26 - 666175905 - Data Validation Failed
